In [1]:
import pandas as pd
import numpy as np 
import re 
from bs4 import BeautifulSoup
import concurrent.futures
import requests

In [2]:
file = r'C:\\Users\\Safwen\\Desktop\\od\\companies_sorted.csv' # Add file name here
df = pd.read_csv(file) # Reading the dataframe from excel

In [17]:
df

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960
...,...,...,...,...,...,...,...,...,...,...,...
7173421,1494427,certiport vouchers,certiportvouchers.com,2011.0,information technology and services,1 - 10,NaN,NaN,linkedin.com/company/certiport-vouchers,0,1
7173422,1494429,black tiger fight club,blacktigerclub.com,2006.0,"health, wellness and fitness",1 - 10,"peking, beijing, china",china,linkedin.com/company/black-tiger-club-hero,0,6
7173423,4768462,catholic bishop of chicago,NaN,NaN,religious institutions,1 - 10,"inverness, illinois, united states",united states,linkedin.com/company/catholic-bishop-of-chicago,0,1
7173424,1494436,medexo robotics ltd,NaN,NaN,research,1 - 10,"london, london, united kingdom",united kingdom,linkedin.com/company/medexo-robotics-ltd,0,2


In [5]:
df2 = df.head(5)

In [6]:
search = []

for word in df2["name"]:

    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    text = word
    url = 'https://google.com/search?q=' + text

    html = requests.get(url,headers=headers)

    soup = BeautifulSoup(html.text, 'html.parser')

    allData = soup.find_all("div",{"class":"g"})

In [ ]:
#Fetching dataframe
search = []

for word in df2["name"]:

    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    text = word
    url = f'https://google.com/search?q=' + text

    html = requests.get(url,headers=headers)

    soup = BeautifulSoup(html.text, 'html.parser')

    allData = soup.find_all("div",{"class":"g"})

    g=0
    Data = [ ]
    l={}
    for i in range(0,len(allData)):
                        link = allData[i].find('a').get('href')

                        if(link is not None):
                            if(link.find('https') != -1 and link.find('http') == 0 and link.find('aclk') == -1):
                                g=g+1
                                l["link"]=link
                                try:
                                    l["title"]=allData[i].find('h3',{"class":"DKV0Md"}).text
                                except:
                                    l["title"]=None

                                try:
                                    l["description"]=allData[i].find("div",{"class":"lVm3ye"}).text
                                except:
                                    l["description"]=None

                                l["position"]=g

                                Data.append(l)

                                l={}

                            else:
                                continue

                        else:
                            continue
    search.append(Data)                    
    print(Data)

In [13]:
# Transform list into a dataframe
result = []
for i in range(len(search)):
    result.append(search[i][0])

df_2 = pd.DataFrame(result)


In [15]:
# Merge original dataframe with their search results
df_result = pd.merge(df, df_2, left_index=True, right_index=True)

In [25]:
#Dataframe cleaning
df_result.drop(["position"],axis=1).to_excel("companies.xlsx")